In [167]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [168]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [169]:
train_data_path = "images/train/"
test_data_path = "images/test/"
val_data_path = "images/val/"

In [170]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])


In [171]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms)

In [172]:
batch_size = 64

In [173]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

In [174]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(12288,84)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(84,50)
        self.relu2 = nn.ReLU()
        self.layer3 = nn.Linear(50,2)
        self.softmax = nn.Softmax()
        
    def forward(self,x):
        x = x.view(-1,12288)
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.layer3(x)
        x = self.softmax(x)
        return x        

In [175]:
model = SimpleNet()

In [176]:
criterion = nn.CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [177]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

model.to(device)

SimpleNet(
  (layer1): Linear(in_features=12288, out_features=84, bias=True)
  (relu1): ReLU()
  (layer2): Linear(in_features=84, out_features=50, bias=True)
  (relu2): ReLU()
  (layer3): Linear(in_features=50, out_features=2, bias=True)
  (softmax): Softmax(dim=None)
)

In [178]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
train(model, optimizer,criterion, train_data_loader,val_data_loader, epochs=10, device=device)